In [134]:
import pandas as pd

### **CLEANING** ###

    EXPLORING THE DATASETS TO SEE WHAT INFO I'VE GOT

In [135]:
df_1 = pd.read_csv("../posibles_datasets/IMDB-Movie-Data.csv")
df_2 = pd.read_csv("../posibles_datasets/Top 100 Greatest Hollywood Actors of All Time.csv")
df_3 = pd.read_csv("../posibles_datasets/amazon_prime_titles.csv")
df_4 = pd.read_csv("../posibles_datasets/database.csv")
df_5 = pd.read_csv("../posibles_datasets/hulu_titles.csv")
df_6 = pd.read_csv("../posibles_datasets/imdb_top_1000.csv")
df_7 = pd.read_csv("../posibles_datasets/netflix_titles.csv")

- DF 1

- DF 2

In [136]:
df_2.head(5)

,Name,Date of Birth,Place of Birth,Oscars,Oscar Nominations,BAFTA,BAFTA Nominations,Golden Globes,Golden Globe Nominations,Greatest Performances
0,Al Pacino,25 April 1940,"East Harlem, New York, United States",1,8,2,5,4,15,"Serpico, Scent of a Woman, Dog Day Afternoon, ..."
1,Alan Arkin,26 March 1934,"Brooklyn, New York, United States",1,4,1,3,1,6,"Argo, Little Miss Sunshine, The Heart Is a Lon..."
2,Alan Rickman,21 February 1946,"Hammersmith, London, United Kingdom",0,0,1,4,1,1,"Michael Collins, Truly Madly Deeply, Sense and..."
3,Albert Finney,9 May 1936,"Charlestown, Salford, United Kingdom",0,5,2,13,2,9,"The Murder on the orient Express, The Dresser,..."
4,Alec Guinness,2 April 1914,"Maida Vale, London, United Kingdom",1,5,3,7,1,3,"Star Wars, Little Dorrit, The Bridge on the Ri..."


- DF 3

- DF 4

In [137]:
df_4.head(5)

,Year,Ceremony,Award,Winner,Name,Film
0,1927/1928,1,Actor,NaN,Richard Barthelmess,The Noose
1,1927/1928,1,Actor,1.0,Emil Jannings,The Last Command
2,1927/1928,1,Actress,NaN,Louise Dresser,A Ship Comes In
3,1927/1928,1,Actress,1.0,Janet Gaynor,7th Heaven
4,1927/1928,1,Actress,NaN,Gloria Swanson,Sadie Thompson


In [138]:
def filtered_data_4(df):

    '''Function to filter and clean df 4.
    Takes 1 arg:
    - df_4
    '''

    df_7 = df[df["Award"]== "Actress"]
    df_1 = df[df["Award"]=="Actor"]
    df_2 = df[df["Award"]=="Actor in a Supporting Role"]
    df_3 = df[df["Award"]=="Actress in a Supporting Role"]
    df_4 = df[df["Award"]=="Actor in a Leading Role"]
    df_5 = df[df["Award"]=="Actress in a Leading Role"]
    df_6 = pd.concat([df_7, df_1, df_2, df_3, df_4, df_5], ignore_index=True)
    df_6['Winner'].fillna(0, inplace=True)
    
    return df_6


In [139]:
df_4 = filtered_data_4(df_4)
df_4

,Year,Ceremony,Award,Winner,Name,Film
0,1927/1928,1,Actress,0.0,Louise Dresser,A Ship Comes In
1,1927/1928,1,Actress,1.0,Janet Gaynor,7th Heaven
2,1927/1928,1,Actress,0.0,Gloria Swanson,Sadie Thompson
3,1928/1929,2,Actress,0.0,Ruth Chatterton,Madame X
4,1928/1929,2,Actress,0.0,Betty Compson,The Barker
...,...,...,...,...,...,...
1663,2015,88,Actress in a Leading Role,0.0,Cate Blanchett,Carol
1664,2015,88,Actress in a Leading Role,1.0,Brie Larson,Room
1665,2015,88,Actress in a Leading Role,0.0,Jennifer Lawrence,Joy
1666,2015,88,Actress in a Leading Role,0.0,Charlotte Rampling,45 Years


- DF 5

- DF 6

- DF 7

        IMPORTING THE DATAFRAMES INTO SQL